In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle

In [83]:
cwd = os.getcwd()
cancel = pd.read_csv(cwd + '/cancel.csv')

In [4]:
infile = open('yearly_flights.pickle', 'rb')
yearly = pickle.load(infile)
infile.close()

"A" = "Carrier" <br> 
> The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, etc.).<br>

"B" = "Weather" <br> 
> Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.<br>

"C" = "National Air System" <br>
> Cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.<br>

"D" = "Security" <br>
> Cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas.<br>

In [84]:
cancel['Freq'] = cancel.groupby(['ORIGIN'])['OP_UNIQUE_CARRIER'].transform('count')
cancel = cancel[cancel['Freq'] > 10] 
#removed any airport used less than 10 times within the year
# went from 110320 rows to 109983 rows

In [102]:
A = cancel[cancel['CANCELLATION_CODE'] == 'A']
B = cancel[cancel['CANCELLATION_CODE'] == 'B']
C = cancel[cancel['CANCELLATION_CODE'] == 'C']
D = cancel[cancel['CANCELLATION_CODE'] == 'D']

## CODE: A

In [94]:
# A has 28085 rows
# 267 unique cities, 273 unique airports

In [5]:
#certain planes that might be more prone to canceltaion
A['TAIL_NUM'].value_counts()[:15]

N719EV    66
N738EV    60
N226JS    52
N248PS    51
N603QX    49
N244PS    48
N250PS    48
N258PS    48
N251PS    47
N617QX    47
N600QX    45
N611QX    44
N230PS    42
N720EV    42
N223JS    41
Name: TAIL_NUM, dtype: int64

In [11]:
A['OP_UNIQUE_CARRIER'].value_counts()

WN    5372
OH    4425
AA    4176
YV    2216
YX    1663
MQ    1519
F9    1476
B6    1377
EV    1195
AS     985
UA     891
OO     772
DL     565
G4     396
NK     331
9E     314
HA     256
VX     224
Name: OP_UNIQUE_CARRIER, dtype: int64

In [19]:
A['MONTH'].value_counts()

6     5970
7     3400
8     3100
3     2355
1     2323
5     2204
9     1882
4     1869
2     1735
10    1641
12     997
11     677
Name: MONTH, dtype: int64

In [104]:
A['Freq'] = A.groupby(['ORIGIN'])['OP_UNIQUE_CARRIER'].transform('count')
A['Freq_month'] = A.groupby(['ORIGIN', 'MONTH'])['OP_UNIQUE_CARRIER'].transform('count')

/Users/arihorwitz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
A100 = A[A['Freq'] > 100] 

In [137]:
Amo = A[A['Freq_month'] > 25]

In [121]:
A100['MONTH'].value_counts()

6     4459
7     2841
8     2636
3     1964
5     1848
1     1780
4     1594
9     1515
2     1366
10    1266
12     849
11     563
Name: MONTH, dtype: int64

In [161]:
# J = A100.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN'])['MONTH'].count()
J = A100.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN']).size()

In [190]:
airline = A100['OP_UNIQUE_CARRIER'].unique()
airport = A100['ORIGIN'].unique()
month = A100['MONTH'].unique()

In [211]:
combo = []
for l in airline:
    for a in airport:
        if a in J[l]:
            if J[l][a] > 100:
                combo.append([J[l][a], l, a])
                        

In [226]:
worst_line_port = sorted(combo, reverse=True)

In [263]:
len(worst_line_port)

50

In [228]:
worst_line_port[:5] 

[[1363, 'OH', 'CLT'],
 [572, 'WN', 'LAX'],
 [498, 'AA', 'DFW'],
 [495, 'YV', 'DFW'],
 [434, 'WN', 'OAK']]

In [218]:
M = A100.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN', 'MONTH']).size()

In [246]:
trio = []
for l in airline:
    for a in airport:
        if a in M[l]:
            for m in range(1,12):
                if m in M[l][a]:
                    if M[l][a][m] > 50:
                        trio.append([M[l][a][m], l, a, m])

In [250]:
worst_line_port_month = sorted(trio, reverse=True)

In [252]:
worst_line_port_month[:5] 

[[1161, 'OH', 'CLT', 6],
 [180, 'OH', 'DCA', 6],
 [147, 'AA', 'DFW', 6],
 [131, 'OH', 'PHL', 6],
 [113, 'YX', 'LGA', 3]]

In [ ]:
# giant glitch with copmuter sytem and PAS airlines ('OH') in Charlotte skewed a lot of dat for cancellation

In [255]:
K = A.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN']).size()

In [261]:
airline = A['OP_UNIQUE_CARRIER'].unique()
airport = A['ORIGIN'].unique()
month = A['MONTH'].unique()

acombo = []
for l in airline:
    for a in airport:
        if a in K[l]:
            if K[l][a] > 100:
                acombo.append([J[l][a], l, a])

In [262]:
len(acombo)

50

In [259]:
len(K)

1419

## CODE: B

## CODE: C

## CODE: D